<a href="https://colab.research.google.com/github/AryanRajSaxena/Leaf_disease/blob/main/Leaves_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.32.0
    Uninstalling tensorflow-io-gcs-filesystem-0.32.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.32.0


In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [ ]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(224,224,3)
)

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
conv_base.trainable = False

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate images by 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the total width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the total height
    shear_range=0.2,  # Apply random shear transformations
    zoom_range=0.2,  # Randomly zoom in and out of images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in any newly created pixels after rotation or shifting
)

NameError: ignored

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Image/train",
    labels="inferred",
    label_mode="int",
    image_size=(224, 224)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory="/content/drive/MyDrive/Image/Val",
    labels="inferred",
    label_mode="int",
    image_size=(224, 224)
)

Found 82 files belonging to 2 classes.
Found 60 files belonging to 2 classes.


In [ ]:
# Normalizing
def process(image, label):
    image = tf.cast(image / 255., tf.float32)
    tfio.experimental.color.bgr_to_rgb(
    image, name=None
    )
    return image, label


train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs=35,validation_data=validation_ds)

Epoch 1/35
3/3 [==============================] - 102s 22s/step - loss: 5.7157 - accuracy: 0.4756 - val_loss: 5.4320 - val_accuracy: 0.5167
Epoch 2/35
3/3 [==============================] - 18s 4s/step - loss: 4.5749 - accuracy: 0.6220 - val_loss: 6.0648 - val_accuracy: 0.5167
Epoch 3/35
3/3 [==============================] - 20s 5s/step - loss: 3.8506 - accuracy: 0.6220 - val_loss: 3.3331 - val_accuracy: 0.5167
Epoch 4/35
3/3 [==============================] - 18s 4s/step - loss: 1.6480 - accuracy: 0.6220 - val_loss: 0.6053 - val_accuracy: 0.6667
Epoch 5/35
3/3 [==============================] - 22s 5s/step - loss: 0.6101 - accuracy: 0.6220 - val_loss: 0.6900 - val_accuracy: 0.4833
Epoch 6/35
3/3 [==============================] - 21s 5s/step - loss: 0.6946 - accuracy: 0.3780 - val_loss: 0.6741 - val_accuracy: 0.6500
Epoch 7/35
3/3 [==============================] - 23s 5s/step - loss: 0.6517 - accuracy: 0.8171 - val_loss: 0.6709 - val_accuracy: 0.5167
Epoch 8/35
3/3 [================

In [ ]:
test_img = r"/content/Green.jpg"
image = cv2.imread(test_img)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)

In [ ]:
image.shape

In [ ]:
image = cv2.resize(image,(256,256))
test_input = image.reshape((256,256,3))

In [ ]:
test_input.shape

In [ ]:
result = model.predict(test_input)
result